## Retrieval Augmented Generation (RAG)

### Instalacja bibliotek

In [4]:
!pip install langchain_text_splitters

### Budowa VectorStore (FAISS) i Retrievera

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

docs = [
    "LangChain to framework do pracy z LLM.",
    "RAG łączy retrieval kontekstu z generacją odpowiedzi.",
    "FAISS to biblioteka do przechowywania i wyszukiwania embeddingów.",
    "Retriever służy do wyszukiwania najbardziej podobnych dokumentów do zapytania użytkownika. Retriever może zwrócić różną liczbę pasujących dokumentów określona w parametrze k. Retriever wykorzystuje różne algorytmy podobieństwa tekstów, np. dopasowanie kosinusowe, odległość euklidesowa, MMR."
]

# Podział dokumentów na fragmenty
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
split = splitter.create_documents(docs)

print(f"Number of chunks: {len(split)}")

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

query = "Po co używa się retrievera?"
context = retriever.invoke(query)
print("Znaleziony kontekst:")
for i, c in enumerate(context, 1):
    print(f"{i}.", c.page_content)

Number of chunks: 7
Znaleziony kontekst:
1. Retriever służy do wyszukiwania najbardziej podobnych dokumentów do zapytania użytkownika. Retriever
2. Retriever może zwrócić różną liczbę pasujących dokumentów określona w parametrze k. Retriever


### Prosty łańcuch RAG (prompt + kontekst + LLM)

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Udziel precyzyjnej odpowiedzi wyłącznie na podstawie KONTEKSTU. Jeśli brak danych — powiedz, że nie wiesz."),
    ("system", "KONTEKST:\n{context}"),
    ("user", "{question}")
])

from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym jest FAISS i do czego służy?"))

FAISS to biblioteka do przechowywania i wyszukiwania embeddingów. Służy do efektywnego wyszukiwania podobnych dokumentów w dużych zbiorach danych.


### Przykład RAG - cały program

In [25]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Dokumenty źródłowe
docs = [
    "LangChain to framework do pracy z LLM.",
    "RAG łączy dopasowanie kontekstu z generacją odpowiedzi.",
    "FAISS to biblioteka do przechowywania i wyszukiwania embeddingów."
]

# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splits = splitter.create_documents(docs)

# Embeddings + vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiadaj tylko na podstawie kontekstu:\n{context}"),
    ("user", "{question}")
])

# Pipeline
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)


print(rag_chain.invoke({"question": "Co to jest FAISS?"}))


FAISS to biblioteka do przechowywania i przeszukiwania dużych zbiorów wektorów.


### RAG z pętlą i ewaluacją

In [28]:
from langchain_core.prompts import ChatPromptTemplate

eval_prompt = ChatPromptTemplate.from_messages([
    ("system", "Oceń odpowiedź."),
    ("user", "Pytanie: {question}\nOdpowiedź: {answer}\nCzy odpowiedź jest poprawna? Odpowiedz yes/no.") #Pytanie "Czy odpowiedź jest kompletna?" spowoduje kolejne iteracje
])

def rag_with_eval(question, max_retries):
    for attempt in range(max_retries):
        context = retriever.invoke(question)
        answer = (prompt | llm | StrOutputParser()).invoke({"context": context, "question": question})
        eval_result = (eval_prompt | llm | StrOutputParser()).invoke({"question": question, "answer": answer})
        print(f"Wynik ewaluacji {eval_result}")
        if "yes" in eval_result.lower():
            return f"✅ Odpowiedź zaakceptowana:\n{answer}"
        print(f"❌ Odpowiedź: {answer}\n odrzucona, ponawiam próbę...")
    return "Nie udało się uzyskać poprawnej odpowiedzi."

print(rag_with_eval("Co to jest RAG?", max_retries=3))


Wynik ewaluacji Yes.
✅ Odpowiedź zaakceptowana:
RAG łączy dopasowanie kontekstu z generacją odpowiedzi.
